In [1]:
import MeCab

def get_iobcorpus(file_name):
    
    # 順番を揃える
    # アノテーション情報用意
    tag_number = []
    tag_label = []
    tag_vocab = {}
    with open(file_name, 'r') as f:
        for i in f.read().split("\n"):
            if len(i) != 0:
                t_s = int(i.split()[2]) # アノテーションの先頭
                t_e = int(i.split()[3]) # アノテーションの末尾
                tag_vocab[t_s] = i.split()[1]
                tag_number.append(t_s)
                tag_number.append(t_e)
    tag_number = sorted(tag_number)
    for k, v in sorted(tag_vocab.items()):
        tag_label.append(v)
    
    # 原文データ用意
    mt = MeCab.Tagger('-Ochasen')
    result = mt.parseToNode("")
    
    file_name = file_name.replace("ann","txt")
     
    word_count = -1 # 単語サイズ
    n = 0 # タグ番号のインデックス
    p = -1 # アノテーションのラベルのインデックス
    
    features_set = []

    with open(file_name, 'r') as f:
        data_text = f.read() # ファイル読み込み
        data_text = data_text.replace("\n",",") # mecabの改行コードが消える部分を補完(「\n」→「,」)
        result = mt.parseToNode(data_text) # 形態素解析
        
        while result:
            
            feature_list = result.feature.split(',')
            
            # 先頭処理
            if feature_list[0] == 'BOS/EOS':
                result = result.next
                continue                        

            features = [] # 特徴            
            features.append(result.surface) # 単語特徴            
            for feature in feature_list: # 形態素特徴
                features.append(feature)
            
            word_count += len(result.surface) # 単語サイズ
            
            # 文末処理
            if n >= len(tag_number):
                features.append("O")
                features_set.append(features)
                result = result.next
                continue 
          
            # タグ処理
            if n%2 == 0:
                if tag_number[n] <= word_count:
                    p += 1
                    features.append("B-{0}".format(tag_label[p]))
                    n += 1
                else:
                    features.append("O")
            elif n%2 != 0:    
                if tag_number[n] <=  word_count:
                    features.append("O")
                    n += 1
                else:
                    features.append("I-{0}".format(tag_label[p]))

            features_set.append(features)
            result = result.next

    return features_set

In [2]:
output_f = open("corpus.txt", "w")
f = "/Users/aharenchie/Desktop/ArticleAnalysis/brat-v1.3_Crunchy_Frog/data/R_data_txt/data1.ann"

output_line = ""
for line in get_iobcorpus(f):
    if line[0] == ",": # mecabの改行コードが消える部分を補完 ( 「,」 → 「 」)
        output_f.writelines(" ") 
    else:
        output_line = "\t".join(line)
        output_f.writelines(output_line+"\n")

output_f.close()

In [3]:
import os

file_list = []
# フォルダーからファイル検索
for file_name in os.listdir("/Users/aharenchie/Desktop/ArticleAnalysis/brat-v1.3_Crunchy_Frog/data/R_data_txt"):
    file_dir = "/Users/aharenchie/Desktop/ArticleAnalysis/brat-v1.3_Crunchy_Frog/data/R_data_txt/{0}".format(file_name)
    if os.path.splitext(file_dir)[1] == ".ann":
        if os.path.getsize(file_dir) > 0:
            file_list.append(file_dir)

# 書き出しファイル準備
output_f = open("corpus.txt", "w")

# ファイル書き出し
for i,f in enumerate(file_list):
    print(i)
    output_line = ""
    for line in get_iobcorpus(f):
        if line[0] == ",": # mecabの改行コードが消える部分を補完 ( 「,」 → 「 」)
            output_f.writelines("\n") 
        else:
            output_line = "\t".join(line)
            output_f.writelines(output_line+"\n")

output_f.close()


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
